In [ ]:
import pandas as pd
import pandas_datareader.data as web
import datetime as dt
import pickle
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')

In [ ]:
start = dt.datetime(2020,1,1)
end = dt.date.today()

In [ ]:
with open('nifty50.pickle','rb') as f:  #TICKERS OF TOP 50 COMPANIES IN NSE
    tickers=pickle.load(f)

In [ ]:
if not os.path.exists('stock_df2'):
    os.makedirs('stock_df2')
    
for ticker in tickers:
    if not os.path.exists('stock_df2/{}.csv'.format(ticker)):
        df= web.DataReader(ticker, 'yahoo', start , end)
        df.to_csv('stock_df2/{}.csv'.format(ticker)) 
        df.rename(columns= {'Adj Close': ticker},inplace=True)
        df.drop(['Open' ,'High', 'Low','Close', 'Volume'],1,inplace=True)

In [ ]:
buy_sell_hold=[]
buy_sell_hold= pd.DataFrame()
for ticker in tickers:
    df= pd.read_csv('stock_df2/{}.csv'.format(ticker))
    df.index= df['Date']
    df['MA21']= df['Adj Close'].rolling(window= 21, min_periods=0).mean()
    df['MA55']= df['Adj Close'].rolling(window= 55, min_periods=0).mean()
    df['signal']= df['MA21']- df['MA55']>0
   
    df['{}'.format(ticker)]=0
    df['EMA_{}' .format(ticker)] =0
    df['DEMA_{}' .format(ticker)] =0
    df['EMPTY_{}'.format(ticker)]= np.nan
    
    df['EMA21']= df['Adj Close']
    df['EMA55']= df['Adj Close']
    
    df['DEMA21']= df['Adj Close']
    df['DEMA55']= df['Adj Close']
    
    for i in range(1,len(df)):
        if i==20:
            df['EMA21'][i]= df['Adj Close'][:20].mean()
        elif i>20:
            df['EMA21'][i]= 2/22*df['Adj Close'][i]+ df['EMA21'][i-1]*20/22
            
        if i==54:
            df['EMA55'][i]= df['Adj Close'][:54].mean()
        elif i>54:
            df['EMA55'][i]= 2/56*df['Adj Close'][i]+ df['EMA55'][i-1]*54/56

        if i==20:
            df['DEMA21'][i]= df['Adj Close'][:20].mean()
        elif i>20:
            df['DEMA21'][i]= 2*df['EMA21'][i] - 2/22*df['EMA21'][i]- df['DEMA21'][i-1]*20/22
            
        if i==54:
            df['DEMA55'][i]= df['Adj Close'][:54].mean()
        elif i>54:
            df['DEMA55'][i]= 2*df['EMA55'][i] - 2/56*df['EMA55'][i]- df['DEMA55'][i-1]*54/56
            
        if (df['DEMA21'][i] -df['DEMA55'][i] >0)==1 and (df['DEMA21'][i-1] -df['DEMA55'][i-1] >0)==0:
            df['DEMA_{}' .format(ticker)][i]=1
        elif (df['DEMA21'][i] -df['DEMA55'][i] >0)==0 and (df['DEMA21'][i-1] -df['DEMA55'][i-1] >0)==1:
            df['DEMA_{}' .format(ticker)][i]=-1
            
        if (df['EMA21'][i] -df['EMA55'][i] >0)==1 and (df['EMA21'][i-1] -df['EMA55'][i-1] >0)==0:
            df['EMA_{}' .format(ticker)][i]=1
        elif (df['EMA21'][i] -df['EMA55'][i] >0)==0 and (df['EMA21'][i-1] -df['EMA55'][i-1] >0)==1:
            df['EMA_{}' .format(ticker)][i]=-1
            
        if df['signal'][i]==1 and df['signal'][i-1]==0:
            df['{}'.format(ticker)][i]= 1
        elif df['signal'][i]==0 and df['signal'][i-1]==1: 
            df['{}'.format(ticker)][i]= -1

    buy_sell_hold= buy_sell_hold.join(df['{}'.format(ticker)][55:] ,  how='outer')
    buy_sell_hold= buy_sell_hold.join(df['EMA_{}'.format(ticker)][55:] , how='outer')
    buy_sell_hold= buy_sell_hold.join(df['DEMA_{}'.format(ticker)][55:] , how='outer')
    buy_sell_hold= buy_sell_hold.join(df['EMPTY_{}'.format(ticker)][55:] , how='outer')


In [ ]:
buy_sell_hold

In [ ]:
if not os.path.exists('buy_sell_hold.csv'):
    buy_sell_hold.to_csv('buy_sell_hold.csv')